# Implementation Of ChatBot Using NLP 

!pip install numpy scikit-learn nltk ------------> installing library

In [ ]:
# Importing libraries
import random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split  # <-- Add this line
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
nltk.download('punkt_tab')
nltk.download('wordnet')

In [ ]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

#  Data Preparation

In [ ]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "Howdy", "What's up", "Greetings", "Good day", "Good morning", "Good afternoon", "Good evening"],
        "responses": ["Hi there!", "Hello!", "Hey!", "Howdy!", "I'm fine, thank you. How can I help you?", "Nothing much. How about you?", "Hello! How can I assist you today?", "Hey! What's on your mind?"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care", "Catch you later", "I'm leaving", "Talk to you soon", "See you", "Have a good day"],
        "responses": ["Goodbye!", "See you later!", "Take care!", "Have a great day!", "Catch you later!", "Bye! Looking forward to our next chat!", "Goodbye! Stay safe!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it", "You're the best", "Much appreciated", "Thanks for your help", "Thanks for the assistance"],
        "responses": ["You're welcome!", "No problem!", "Glad I could help!", "Happy to assist you!", "Anytime!", "You're welcome! If you need more help, just ask!", "No worries!"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What's your purpose", "What is your role", "Tell me about yourself", "What services do you offer"],
        "responses": ["I am a chatbot designed to assist you with various tasks.", "My purpose is to help you by answering questions and providing information.", "I can chat with you, answer questions, and help you with whatever you need.", "I'm here to make your life easier by providing quick and helpful answers."]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do", "I need assistance", "Can you assist me", "I need some guidance", "I'm stuck, can you help?"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?", "Feel free to ask for help. I'm here for you!", "I'd be glad to help you. What's on your mind?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age", "When were you created", "Are you old or young", "Do you have an age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me.", "I exist beyond time, so no age for me!", "I'm as old as the code that runs me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today", "Can you check the weather", "Tell me the weather forecast", "Is it sunny today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website.", "I can't check the weather, but try using a weather app.", "Sorry, I don't have weather updates. Check online for accurate info."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget", "Tips for managing finances", "Help me with budgeting"],
        "responses": ["To make a budget, track your income and expenses. Allocate for essentials, savings, and leisure.", "Start by listing your income and expenses, then categorize them to prioritize your spending.", "A good strategy is to follow the 50/30/20 rule: 50% needs, 30% wants, and 20% savings.", "Begin by tracking all expenses, then set realistic limits for each category."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score", "Why is credit score important", "Explain credit scores"],
        "responses": ["A credit score represents your creditworthiness based on your credit history.", "To check your credit score, use authorized credit bureaus or financial apps.", "Paying bills on time and reducing debt can improve your credit score.", "A high credit score increases your chances of loan approvals and better interest rates."]
    },
    {
        "tag": "developer",
        "patterns": ["Who is your developer", "Who created you", "Who made you", "Who built this chatbot", "Who programmed you"],
        "responses": ["My developer is Suraj Sharma.", "I was created by Suraj Sharma.", "Suraj Sharma built me.", "I was programmed by Suraj Sharma."]
    },
    {
        "tag": "play_music",
        "patterns": ["Hey, can you play a song for me", "Please play a song", "Play music", "Can you play some music", "I want to listen to music"],
        "responses": ["Sure! Which song would you like to hear?", "Yes! Let me know your favorite song.", "I'd love to! Tell me the song name.", "Which track are you in the mood for?", "I can play music! What's your favorite genre?"]
    },
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Do you know any jokes", "Cheer me up with a joke", "Say something funny"],
        "responses": ["Why did the computer break up with the internet? It felt disconnected!", "I would tell you a joke about an elevator, but it's an uplifting experience!", "Why did the smartphone need glasses? It lost its contacts!", "I told my computer I needed a break, and now it won't stop sending me Kit Kats!"]
    },
    {
        "tag": "time",
        "patterns": ["What's the time", "Can you tell me the current time", "What time is it now", "Do you know the time", "Tell me the time"],
        "responses": ["I'm not equipped to provide the current time, but you can check your device!", "I don't have a clock, but your phone does!", "Sorry, I can't tell the time. Maybe check your watch?", "I can't see a clock, but your device should have the answer."]
    },
    {
        "tag": "news",
        "patterns": ["What's the latest news", "Any news updates", "Tell me the news", "What's happening around the world", "Give me some news"],
        "responses": ["I can't fetch live news, but you can check reliable news websites.", "I'm not connected to news services, but I can chat with you!", "I don't have live news, but feel free to discuss any topic with me."]
    }
]


In [ ]:
# Extracting data
patterns = []
tags = []
responses = {}

In [ ]:
for intent in intents:
    for pattern in intent["patterns"]:
        patterns.append(pattern)
        tags.append(intent["tag"])
    responses[intent["tag"]] = intent["responses"]

In [ ]:
# Text Preprocessing
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())  # Tokenize and convert to lowercase
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize words
    return ' '.join(tokens)

In [ ]:
# Applying preprocessing
processed_patterns = [preprocess_text(pattern) for pattern in patterns]

In [ ]:
# Feature Extraction using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 2))
X = vectorizer.fit_transform(processed_patterns).toarray()
y = np.array(tags)

In [ ]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data prepared successfully!")

# Model Building and Training.

In [ ]:
# Choosing the model
model = LogisticRegression(max_iter=200)

In [ ]:
# Training the model
model.fit(X_train, y_train)

In [ ]:
# Evaluating the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

--->>output of above cell : 

Classification Report:
                         precision    recall  f1-score   support

         DIY_home_decor       0.50      1.00      0.67         1
                 advice       0.00      0.00      0.00         1
                animals       0.00      0.00      0.00         2
              astrology       0.00      0.00      0.00         1
       business_startup       0.00      0.00      0.00         1
          career_advice       1.00      1.00      1.00         1
  coding_best_practices       0.00      0.00      0.00         0
       coding_resources       0.00      0.00      0.00         1
          coffee_lovers       1.00      1.00      1.00         2
           cooking_tips       0.00      0.00      0.00         1
       creative_writing       0.00      0.00      0.00         0
     daily_affirmations       0.00      0.00      0.00         1
      digital_marketing       0.00      0.00      0.00         2
           diy_projects       0.00      0.00      0.00         1
         fashion_advice       0.00      0.00      0.00         1
     financial_literacy       0.00      0.00      0.00         2
     financial_planning       1.00      1.00      1.00         1
 gaming_recommendations       1.00      1.00      1.00         1
                goodbye       1.00      1.00      1.00         1
               greeting       0.05      1.00      0.10         1
handwriting_improvement       0.00      0.00      0.00         2
...
               accuracy                           0.34        87
              macro avg       0.29      0.31      0.27        87
           weighted avg       0.38      0.34      0.33        87